In [1]:


import requests as r
from bs4 import BeautifulSoup as bs
import json 
from pymongo import *

url_base = 'http://www.ipeen.com.tw'

In [2]:
#爬取愛評網各個縣市分類的餐廳評論頁數網址
def page_crawler():
#     city_list = ['keelung','taipei','xinbei','taoyuan','hsinchu','hsinchucounty','miaoli','taichung','nantou','changhua','yunlin','chiayi'
#                 ,'chiayicounty','tainan','kaohsiung','pintung','ilan','hualian','taitung','penghu','lianjiang','kinmen']
    
    
    
    #台灣餐廳評論的所有頁數從最新到最舊
    
    try:
        url = 'http://www.ipeen.com.tw/search/taiwan/000/1-0-0-0/'
        res = r.get(url)
        res.encoding="utf-8"
        soup = bs(res.text, 'lxml')
        #取得最後一頁的頁數
        finalpage = soup.select('.next_p_s > a')[0].get('href').split('=')[1]
        int_page = int(finalpage)
        for i in range(1,int_page+1):
            page_url = url+'?p={}'.format(i)
            print(page_url)
            #呼叫取得餐廳裡的所有評論網址的方法
            all_restaurant_list(page_url)
    #避免遇到index error
    except IndexError:
        print("[ERROR] : index error!")
        

In [7]:
#爬取所有餐廳list，並看該餐廳最大的評論數目
def all_restaurant_list(url):
    res = r.get(url)
    res.encoding="utf-8"
    soup = bs(res.text, 'lxml')
    
    
    
    #每頁最多有15個餐廳list
    last_list = soup.select('.serShop')[-1].select('h3')[0].get('id').split('_')[2]
    int_list = int(last_list)
    
    for i in range(1,int_list+1):
        try:
            #取到的是/shop/數字
            restaurant_num = soup.select('#shop_h3_{} > a'.format(i))[0].get('href').split('-')[0]
            print(restaurant_num)
            #取得每間餐廳評論的最大數目
            restaurant_url = 'http://www.ipeen.com.tw/shop/{}/comments?p=1&sortway=d&so=shop_default'.format(restaurant_num.split('/')[2])
            res_r = r.get(restaurant_url)
            res_r.encoding="utf-8"
            soup_r = bs(res_r.text, 'lxml')
            total_share = soup_r.select('h2.main-title')[0].text.split('(')[1].split(')')[0]
            int_share = int(total_share)
            int_page=0
            if int_share%5==0:
                int_page=int_share/5
            else:
                int_page=int_share/5+1
            for i in range(1,int_page+1):
                
                share_page = '/comments?p={}&sortway=d&so=shop_default'.format(i)
                share_url = url_base + restaurant_num + share_page
                print(share_url)
                restaurant_share(restaurant_url)
        except IndexError:
            print("[ERROR] : no page!")
    

In [4]:
#爬取單一餐廳的所有食記網址
def restaurant_share(url):
    res = r.get(url)
    res.encoding="utf-8"
    soup = bs(res.text, 'lxml')

    total_list=[]
    comment_list = soup.find_all('a',{'itemprop':'discussionUrl url'})
    for comment in comment_list:
        comment_url = comment.get('href')
        c_url = url_base + comment_url
#         print(c_url)
        dicts = comment_crawler(c_url)
        total_list.append(dicts)
    goto_mongo(total_list)

In [12]:
restaurant_share('http://www.ipeen.com.tw/shop/67012/comments?p=3&sortway=d&so=shop_default')

{'web_url': 'http://www.ipeen.com.tw/comment/902978', 'open_time': ['11:30~16:00、', '17:30~22:00'], 'restaurant_location': '台北市中山區南京西路12巷13弄9號', 'img_url': ['http://iphoto.ipeen.com.tw/images/v2/service/seal/confused_smile.gif', 'http://iphoto.ipeen.com.tw/images/v2/service/seal/sad_smile.gif', 'http://iphoto.ipeen.com.tw/photo/comment/1259892/902978/cm20150819___6a4a93d99811f576534ff9bf4c722fdc685.jpg', 'http://iphoto.ipeen.com.tw/photo/comment/1259892/902978/cm20150819___028094f4fe538ba2a20d3ef36884f931635.jpg', 'http://iphoto.ipeen.com.tw/photo/comment/1259892/902978/cm20150819___2236145b58fa4339eb8a8ee18f1c7005248.jpg', 'http://iphoto.ipeen.com.tw/images/v2/service/seal/regular_smile.gif', 'http://iphoto.ipeen.com.tw/photo/comment/1259892/902978/cm20150819___db6fa5e149d8dc589faed4dadc86ca1c276.jpg', 'http://iphoto.ipeen.com.tw/images/v2/service/seal/omg_smile.gif', 'http://iphoto.ipeen.com.tw/photo/comment/1259892/902978/cm20150819___088b17ce7db87217c8bed67b86429282733.jpg', 'http:

In [10]:
#爬取食記裡所需要的資料
def comment_crawler(url):
    
#     total_list=[]
   
    try: 
#         url = 'http://www.ipeen.com.tw/comment/{}'.format(i)
        res = r.get(url)
        res.encoding="utf-8"
        soup = bs(res.text, 'lxml')
        mydict = {}
        #餐廳名稱
        restaurant_name = soup.select('.brief > p > a')[0].text.strip()
        mydict['restaurant_name'] = restaurant_name

        #餐廳地址
        restaurant_location = soup.select('.info > ul > li')[2].text.split('：')[1].split('(')[0].strip()
        mydict['restaurant_location'] = restaurant_location

        #電話號碼
        phone_number = soup.select('.info > ul > li')[1].text.split('：')[1]
        mydict['phone_number'] = phone_number

        #發表日期
        created_date = soup.select('.date > span')[0].text
        mydict['created_date'] = created_date

        #營業時間
        open_time_list=[]

        open_time = soup.select('.businessHour-right > span')
        for one_time in open_time:
            open_time_list.append(one_time.text)
        mydict['open_time'] = open_time_list    

        #平均消費
        avg_consume = soup.select('.other > ul > li')[3].text.split('：')[1].strip().split(' ')[0]
        mydict['avg_consume'] = avg_consume

        #url
        mydict['web_url'] = url

        #美味度
        # deli = soup.select('.rating > dt')
        # for d in deli:
        #     n = deli.index(d)

        delicious = soup.select('.rating > dd')[0].text
        mydict['delicious'] = delicious

        #服務品質
        service_quality = soup.select('.rating > dd')[1].text
        mydict['service_quality'] = service_quality

        #環境氣氛
        dining_environment = soup.select('.rating > dd')[2].text
        mydict['dining_environment'] = dining_environment

        #作者名稱
        author_name = soup.find_all('a',{'data-action':'header_user'})[1].select('span')[0].text
        mydict['author_name'] = author_name

        #作者url
        a_url = soup.find_all('a',{'data-action':'header_user'})[0].get('href').split('../')[1]
        author_url = url + '/' + a_url
        mydict['author_url'] = author_url
       
        #種類
        category = soup.find_all('span',{'itemprop':'title'})[4].text
        mydict['category'] = category
        
        #餐廳評論
        comment = soup.select('.description')[0].text.strip()
        mydict['comment'] = comment
       
        
        #圖片url
        image_list = []
        images = soup.select('.description')[0].find_all('img')
        for image in images:
            image_url = image.get('src')
            image_list.append(image_url)
        mydict['img_url'] =  image_list   
        
        print(mydict)
#         total_list.append(mydict)
        #寫進json檔
#         with open('ipeen.json', 'w',encoding='utf8') as f:
#             data = json.dumps(total_list, ensure_ascii=False)
#             f.write(data)
        #把每個dictionary都回傳給上個function並裝成list
        return mydict
    except IndexError: 
        print("[ERROR] : empty comment!")
           

In [11]:
#寫進mongoDB裡面
def goto_mongo(lists):
    client = MongoClient()
    db = client['test']
    collect = db['test_ipeen']
    #一次insert一整個list
    collect.insert_many(lists)

In [31]:
with open('ipeen.json', 'r',encoding='utf8') as a:
    data = json.load(a)
    
    print(data)

[{'open_time': ['12:00~23:00'], 'comment': '年初四很多服務業都開工了，知名"鬆餅下午茶"餐廳『咖啡弄』也在今天開工，沙拉就來跟大家湊個熱鬧開工大吉『咖啡弄(忠孝店)』位在捷運忠孝復興站步行約5分鐘交通便利，11:45到門口阿娘威這人會不會太多了，12點才開門就已經這麼多人在門口等，越到時間人越聚越多，是說這裡沒店吃了嗎?這麼多人要吃"咖啡弄"真的是做口碑的等了15分鐘就開始唱名進場，另一邊迅速的坐滿，所以沙拉只拍了自己坐的這邊，走道還算寬敞，不會相戶碰撞，昏黃的燈光讓人覺得溫馨舒服\xa0翻咖啡弄的菜單就像翻雜誌一樣時尚，而且不時的還有新菜單，來這裡就是要吃新菜色既然是吃正餐，我相中了分店限定的"鼓汁排骨飯"台港美食交流的"鼓汁排骨飯"是分店限定，也就是說只有在"忠孝店"、"士林店"才能吃到"鼓汁排骨飯"排骨本身軟中帶有嚼勁，尾韻帶有微辣喜歡吃辣的沙拉連同排骨和辣椒一起吃，辣椒本身後勁有些強，所以沒吃很辣的不建議學我這樣吃喔!單吃肉就很好吃了以簡餐的飯量來說其實我覺得蠻足的，除了排骨外還可以把底下的汁拿來淋在飯上很下飯喔!墨爾本龍蝦堡小小瓶的油醋可淋在沙拉上，酸酸的好開胃另提供鹽及胡椒可撒在薯塊，薯塊本身沒有任何油脂很爽口黃檸檬可以擠在龍蝦上，蝦醬可以拿來沾麵包，一切都是這麼完美龍蝦給的很大方，可別以為只有外面你看到的，連裡面都塞滿了龍蝦肉，料給的很大方，都爆出來了龍蝦熬的肉吃起來非常紮實，吃到這麼高級的漢堡後回不去了草莓冰淇淋鬆餅本來是想點新品莓果熱蛋糕，但今天開工第一天沒有原物料可以做改點千層酥，開店不到半小時就全數賣完，是說我有這麼好運嗎?應該去買張樂透了買不到想吃的，只好點了『咖啡弄』一向出名的鬆餅滿滿的草莓底下有一球草莓冰淇淋和一大球卡士達鬆餅很熱一上桌草莓馬上滑落露出浪漫粉的草莓冰淇淋烤的香酥的鬆餅一直是沙拉所喜歡的，吃下午茶時總是少不了他，外酥內容越嚼越有麵香新品：牛奶冰棒可可牛奶冰棒可可還附兩塊巧克力當甜點\xa0這飲料真是太可愛了，自製濃郁的牛奶冰棒再淋上熱可可變成了可可牛奶，好喝又好玩桂圓紅棗茶，雖然過年這幾天每天都好天氣，但還是不忘要補氣一下除了整壺滿滿的桂圓枸杞外還有三顆桂圓乾，好久沒吃了回味一下咖啡弄的飲料從沒讓人失望過全餐吃下來兩主餐加一個甜點真的是非常飽也非常滿足地址：台北市大安區忠孝東路四段